In [1]:
from sklearn import datasets
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymysql

## 取得資料

In [24]:
def get_data(table):
    config = {"host" : "104.155.221.250", "port" : 3306, "user" : "sikei",
          "passwd" : "zxcv1234", "db" : "AQI", "charset" : "utf8mb4"}

    conn = pymysql.connect(**config) ## **會將字典型態轉變(kwargs)
    cursor = conn.cursor()
    
    SQL = "select * from {}".format(table)
    print('資料筆數 :',cursor.execute(SQL))
    
    if table == "AQI_day":
        results = cursor.fetchall()
        data = pd.DataFrame(results, columns = ["id", "SiteName", "MonitorDate", "AQI"])
        data = data.drop("id", axis=1)
        
    elif table == "AQI_hour":
        results = cursor.fetchall()
        data = pd.DataFrame(results, columns = ["id", "SiteName", "County", "AQI", "SO2", "CO", "O3", "PM10", "PM25",
                                                "NO2", "NOx", "NO", "WindSpeed", "WindDirec", "DataCreationDate"])
        data = data.drop("id", axis=1)
        
    else:
        results = cursor.fetchall()
        data = pd.DataFrame(results)

    # 關閉連線
    cursor.close()
    conn.close()
    
    return data

In [26]:
df = get_data("AQI_hour")
df

資料筆數 : 24903


,SiteName,County,AQI,SO2,CO,O3,PM10,PM25,NO2,NOx,NO,WindSpeed,WindDirec,DataCreationDate
0,大園,桃園市,34.0,5.00,0.42,16.60,11.00,10.00,23.90,24.40,0.50,1.30,249.0,2021-06-01 00:00:00
1,中壢,桃園市,32.0,1.40,0.78,8.50,11.00,6.00,25.60,33.90,8.30,0.40,327.0,2021-06-01 00:00:00
2,龍潭,桃園市,22.0,1.70,0.29,13.70,10.00,6.00,10.20,12.00,1.80,0.40,263.0,2021-06-01 00:00:00
3,平鎮,桃園市,22.0,2.40,0.50,16.20,10.00,4.00,15.90,16.80,0.90,0.50,358.0,2021-06-01 00:00:00
4,觀音,桃園市,33.0,6.50,0.45,8.90,26.00,14.00,28.70,30.20,1.60,0.40,65.0,2021-06-01 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24898,大園,桃園市,30.0,1.80,0.15,38.40,20.00,5.00,10.00,13.70,3.60,6.60,67.0,2021-11-25 15:00:00
24899,觀音,桃園市,36.0,0.90,0.13,41.70,25.00,4.00,5.60,6.80,1.20,6.90,47.0,2021-11-25 15:00:00
24900,平鎮,桃園市,30.0,2.50,0.34,27.80,30.00,8.00,16.60,21.20,4.60,2.10,54.0,2021-11-25 15:00:00
24901,龍潭,桃園市,30.0,None,None,28.90,None,None,None,None,None,4.40,49.0,2021-11-25 15:00:00


In [30]:
df.isna().sum()
df1 = df.dropna()

In [32]:
X = df1.iloc[:,[3,4,5,8,9,10,11,12]]
y_pm25 = df1.iloc[:,7]
y_pm10 = df1.iloc[:,6]

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y_pm25, test_size = 0.3)

In [34]:
rbfModel = SVR(kernel='rbf', C = 1e3, gamma = "auto")
# 利用訓練集讓模型進行學習
rbfModel.fit(X_train, y_train)
y_pred = rbfModel.predict(X_test)
rbfModel.score(X_test, y_pred)

1.0

## 使用 joblib.dump 匯出預測器
> 將訓練模型儲存起來，下次要使用時就不用再重新訓練一次

In [35]:
joblib.dump(rbfModel,"./machine_SVR.pkl")

['./machine_SVR.pkl']

## 訓練及分類

In [36]:
# 將 pkl 檔匯入為一個 SVR 預測器
rbfModel = joblib.load("./machine_SVR.pkl")
rbfModel.fit(X, y_pm25)
predict_y=rbfModel.predict(X)

## 使用score計算準確率

In [37]:
predict=rbfModel.predict(X)
rbfModel.score(X, y_pm25)

0.9997393189891322